# Store

> Module for constructing AppletStore.

In [ ]:
#| default_exp store

In [ ]:
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import yaml
import json
import os
from importlib import import_module
from llmcam.oas_to_requests import toolbox_schema

In [ ]:
#| export
ToolBox = []

In [ ]:
#| export
def load_oas(
    oas_url: str = "https://tie.digitraffic.fi/swagger/openapi.json",  # OpenAPI Specification URL
    destination: str = "api/road_digitraffic.json",  # Destination file
    overwrite: bool = False  # Overwrite existing file
) -> dict:  # OpenAPI Specification
    """Load OpenAPI Specification from URL or file."""
    # Create destination directory if it does not exist
    os.makedirs(os.path.dirname(destination), exist_ok=True)

    # Download OpenAPI Specification if it does not exist or overwrite is True
    if not os.path.exists(destination) or overwrite:
        r = requests.get(oas_url)
        with open(destination, "w") as f:
            f.write(r.text)

    # Load OpenAPI Specification
    with open(destination, "r") as f:
        if destination.endswith(".json"):
            return json.load(f)
        elif destination.endswith(".yaml") or destination.endswith(".yml"):
            return yaml.load(f)
        else:
            raise ValueError("Invalid file format")

Usage to download three DigiTraffic endpoints

In [ ]:
#| export
road_digitraffic = load_oas(
    oas_url="https://tie.digitraffic.fi/swagger/openapi.json",
    destination="api/road_digitraffic.json",
    overwrite=False
)
train_digitraffic = load_oas(
    oas_url="https://rata.digitraffic.fi/swagger/openapi.json",
    destination="api/train_digitraffic.json",
    overwrite=False
)
marine_digitraffic = load_oas(
    oas_url="https://meri.digitraffic.fi/swagger/openapi.json",
    destination="api/marine_digitraffic.json",
    overwrite=False
)

In [ ]:
road_digitraffic

{'openapi': '3.0.1',
 'info': {'title': 'Digitraffic Road API',
  'description': '[OpenAPI document](/swagger/openapi.json) \n\nDigitraffic is a service operated by the [Fintraffic](https://www.fintraffic.fi) offering real time traffic information. Currently the service covers *road, marine and rail* traffic. More information can be found at the [Digitraffic website](https://www.digitraffic.fi/) \n\nThe service has a public Google-group [road.digitraffic.fi](https://groups.google.com/forum/#!forum/roaddigitrafficfi) for communication between developers, service administrators and Fintraffic. The discussion in the forum is mostly in Finnish, but you\'re welcome to communicate in English too. \n\n### General notes of the API\n* Many Digitraffic APIs use GeoJSON as data format. Definition of the GeoJSON format can be found at https://tools.ietf.org/html/rfc7946.\n* For dates and times [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601) format is used with "Zulu" zero offset from UTC unless

Test functions with GPT:

In [ ]:
import textwrap
from llmcam.fn_to_fc import complete, form_msg
from llmcam.oas_to_requests import generate_request

In [ ]:
#| eval: false
messages = [
    form_msg("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    form_msg("user", "Get me information about the trains departing on 2017-11-09 with train number 1.")
]
complete(messages, toolbox_schema("https://rata.digitraffic.fi", train_digitraffic), aux_fn=generate_request)
for message in messages:
    print(f">> {message['role'].capitalize()}:")
    try:
        print(textwrap.fill(message["content"], 100))
    except:
        print(message)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get me information about the trains departing on 2017-11-09 with train number 1.
>> Assistant:
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_WyALfBABJY1UNmWajdbgOutw', 'function': {'arguments': '{"departure_date":"2017-11-09","train_number":1}', 'name': 'getTrainByTrainNumberAndDepartureDate'}, 'type': 'function'}]}
>> Tool:
{"departure_date": "2017-11-09", "train_number": 1, "getTrainByTrainNumberAndDepartureDate":
[{"trainNumber": 1, "departureDate": "2017-11-09", "operatorUICCode": 10, "operatorShortCode": "vr",
"trainType": "IC", "trainCategory": "Long-distance", "commuterLineID": "", "runningCurrently":
false, "cancelled": false, "version": 231593807888, "timetableType": "REGULAR",
"timetableAcceptanceDate": "2017-07-21T11:27:05.000Z", "timeTableRows": [{"stationShortCode": "HKI",
"stationUICCode": 1, "countryCode": "FI", "type": "DEPARTURE", "

In [ ]:
#| eval: false
messages = [
    form_msg("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    form_msg("user", "Are there any active nautical warnings?")
]
complete(messages, toolbox_schema("https://meri.digitraffic.fi", marine_digitraffic), aux_fn=generate_request)
for message in messages:
    print(f">> {message['role'].capitalize()}:")
    try:
        print(textwrap.fill(message["content"], 100))
    except:
        print(message)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Are there any active nautical warnings?
>> Assistant:
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_S3mAaovBQjkeNDjSlsbbk95q', 'function': {'arguments': '{"url":"https://meri.digitraffic.fi/api/nautical-warning/v1/warnings/active","method":"get"}', 'name': 'get_api_nautical_warning_v1_warnings_active'}, 'type': 'function'}]}
>> Tool:
{"url": "https://meri.digitraffic.fi/api/nautical-warning/v1/warnings/active", "method": "get",
"get_api_nautical_warning_v1_warnings_active": {"type": "FeatureCollection", "features": [{"type":
"Feature", "geometry": {"type": "Polygon", "coordinates": [[[23.35803893344, 64.1366363392165],
[23.938027213478897, 64.4119896036909], [24.1078447347889, 64.58443222591849], [23.9197195479886,
64.7294449566291], [23.6817198966139, 64.8013598139946], [23.357787405160398, 64.8191679117966],
[22.9866304792208, 64.72628481643409], 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()